<a href="https://colab.research.google.com/github/royam0820/nanochat_rl/blob/main/nanochat_1_8B_rl_inference_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NanoChat 1.8B-RL Inference Notebook

This notebook demonstrates how to run inference with the `jasonacox/nanochat-1.8B-rl` model, a 1.9B parameter GPT-style transformer trained from scratch using Andrej Karpathy's NanoChat framework.

**Model Details:**
- Parameters: ~1.9 billion
- Architecture: 20 layers, 1280 embedding dimension
- Training: Includes pretraining, midtraining, SFT, and RL (GRPO)
- Use Case: Educational purposes, experimentation, conversational AI

**Author:** Adapted for Colab inference

**Model Link:** [jasonacox/nanochat-1.8B-rl](https://huggingface.co/jasonacox/nanochat-1.8B-rl)

## 1. Setup and Installation

First, we'll check GPU availability and install the necessary dependencies.

In [ ]:
# Check GPU availability
!nvidia-smi

Wed Nov 19 18:07:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Install dependencies
!pip install -q huggingface_hub torch transformers

## 2. Clone NanoChat Repository

We need the NanoChat codebase to properly load and run the model.

In [ ]:
import os
import sys

# Clone the NanoChat repository if not already present
if not os.path.exists('nanochat'):
    !git clone https://github.com/karpathy/nanochat.git
    print("✓ NanoChat repository cloned successfully")
else:
    print("✓ NanoChat repository already exists")

# Add to Python path
sys.path.insert(0, 'nanochat')

Cloning into 'nanochat'...
remote: Enumerating objects: 629, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 629 (delta 25), reused 10 (delta 10), pack-reused 585 (from 3)
Receiving objects: 100% (629/629), 430.77 KiB | 1.95 MiB/s, done.
Resolving deltas: 100% (394/394), done.
✓ NanoChat repository cloned successfully


## 3. Download Model from Hugging Face

Download the nanochat-1.8B-rl model files from Hugging Face Hub.

In [ ]:
from huggingface_hub import snapshot_download

# Download model from HuggingFace
print("Downloading nanochat-1.8B-rl model...")
model_path = snapshot_download(
    repo_id="jasonacox/nanochat-1.8B-rl",
    cache_dir=os.path.expanduser("~/.cache/nanochat/hf_downloads")
)

print(f"✓ Model downloaded to: {model_path}")

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

meta_000466.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/248 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer/tokenizer.pkl:   0%|          | 0.00/846k [00:00<?, ?B/s]

model_000466.pt:   0%|          | 0.00/2.08G [00:00<?, ?B/s]

tokenizer/token_bytes.pt:   0%|          | 0.00/264k [00:00<?, ?B/s]

✓ Model downloaded to: /root/.cache/nanochat/hf_downloads/models--jasonacox--nanochat-1.8B-rl/snapshots/53460595bc0c4ff1e31df01c59711798a372bd6a


## 4. Organize Model Files

The NanoChat framework expects files in specific directories. Let's organize them properly.

In [ ]:
import shutil
import os

# Create the expected directory structure
base_cache_dir = os.path.expanduser("~/.cache/nanochat")
tokenizer_dir = os.path.join(base_cache_dir, "tokenizer")
checkpoint_dir = os.path.join(base_cache_dir, "rl_checkpoints", "d20")

os.makedirs(tokenizer_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

print("Organizing model files...")

# List all files in the downloaded model
all_files = os.listdir(model_path)
print(f"\nFiles found: {all_files}")

# Copy tokenizer files from the 'tokenizer' subdirectory
tokenizer_source_dir = os.path.join(model_path, 'tokenizer')
if os.path.exists(tokenizer_source_dir) and os.path.isdir(tokenizer_source_dir):
    for file_name in os.listdir(tokenizer_source_dir):
        src = os.path.join(tokenizer_source_dir, file_name)
        dst = os.path.join(tokenizer_dir, file_name)
        shutil.copy2(src, dst)
        print(f"✓ Copied {file_name} to tokenizer directory")
else:
    print(f"⚠ Warning: Tokenizer directory not found at {tokenizer_source_dir}")

# Copy model checkpoint files
checkpoint_files = [f for f in all_files if f.startswith('model_') or f.startswith('meta_')]
for file in checkpoint_files:
    src = os.path.join(model_path, file)
    dst = os.path.join(checkpoint_dir, file)
    shutil.copy2(src, dst)
    print(f"✓ Copied {file} to checkpoint directory")

print(f"\n✓ Files organized successfully!")
print(f"  Tokenizer dir: {tokenizer_dir}")
print(f"  Checkpoint dir: {checkpoint_dir}")

# Verification step: Check if tokenizer.pkl exists in the target directory
tokenizer_pkl_path = os.path.join(tokenizer_dir, "tokenizer.pkl")
if os.path.exists(tokenizer_pkl_path):
    print(f"✓ Verification: tokenizer.pkl found at {tokenizer_pkl_path}")
else:
    print(f"✗ Verification: tokenizer.pkl NOT FOUND at {tokenizer_pkl_path}")
    print("This indicates a problem with the file copy or persistence.")

Organizing model files...

Files found: ['config.json', 'model_000466.pt', 'tokenizer', '.gitattributes', 'meta_000466.json', 'README.md']
✓ Copied tokenizer.pkl to tokenizer directory
✓ Copied token_bytes.pt to tokenizer directory
✓ Copied model_000466.pt to checkpoint directory
✓ Copied meta_000466.json to checkpoint directory

✓ Files organized successfully!
  Tokenizer dir: /root/.cache/nanochat/tokenizer
  Checkpoint dir: /root/.cache/nanochat/rl_checkpoints/d20


## 5. Load Model and Initialize Engine

Load the model checkpoint and create an inference engine.

In [ ]:
import glob
import torch
from contextlib import nullcontext

# Import NanoChat modules
from nanochat.checkpoint_manager import build_model
from nanochat.common import compute_init, autodetect_device_type
from nanochat.engine import Engine

# Initialize device
device_type = autodetect_device_type()
print(f"Using device: {device_type}")

_, _, _, _, device = compute_init(device_type)

# Set precision
ptdtype = torch.bfloat16 if device_type == "cuda" else torch.float32
autocast_ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype) if device_type == "cuda" else nullcontext()

# Find checkpoint file in the organized directory
checkpoint_files = glob.glob(os.path.join(checkpoint_dir, "model_*.pt"))
if not checkpoint_files:
    raise FileNotFoundError(f"No checkpoint files found in {checkpoint_dir}")

step = int(os.path.basename(checkpoint_files[0]).split("_")[-1].split(".")[0])
print(f"Loading checkpoint from step {step}...")

# Load model using the checkpoint directory
model, tokenizer, _ = build_model(checkpoint_dir, step, device, phase="eval")
engine = Engine(model, tokenizer)

print("✓ Model loaded successfully!")
print(f"✓ Model has ~1.9B parameters")

Autodetected device type: cuda
Using device: cuda
Loading checkpoint from step 466...
✓ Model loaded successfully!
✓ Model has ~1.9B parameters


## 6. Simple Inference Function

Create a helper function to generate responses from the model.

In [ ]:
def generate_response(prompt, max_tokens=200, temperature=0.8, top_k=50, verbose=True):
    """
    Generate a response from the nanochat model.

    Args:
        prompt (str): Input text prompt
        max_tokens (int): Maximum number of tokens to generate
        temperature (float): Sampling temperature (0.0 to 1.0+)
        top_k (int): Top-k sampling parameter
        verbose (bool): Print tokens as they're generated

    Returns:
        str: Generated response
    """
    # Encode the prompt
    tokens = tokenizer.encode(prompt)

    if verbose:
        print(f"Prompt: {prompt}")
        print(f"Response: ", end="", flush=True)

    response_tokens = []

    # Generate tokens
    with autocast_ctx:
        for token_column, _ in engine.generate(
            tokens,
            num_samples=1,
            max_tokens=max_tokens,
            temperature=temperature,
            top_k=top_k
        ):
            token = token_column[0]
            response_tokens.append(token)

            if verbose:
                print(tokenizer.decode([token]), end="", flush=True)

    if verbose:
        print("\n")

    # Decode full response
    response = tokenizer.decode(response_tokens)
    return response

## 7. Example Usage

Let's test the model with various prompts!

In [15]:
# Example 1: Simple greeting
response = generate_response(
    "Hello! How are you today?",
    max_tokens=100,
    temperature=0.8
)

Prompt: Hello! How are you today?
Response:  I'm at the store, I have to buy some milk. Now let me tell you what I think: I like chocolate. I think I like milk too. That's all for now. Hope you enjoy your day. How about this? Oh, I see some flowers in the window. The sunlight   is really bright today. Today is a nice day. We should go to the park to have a rest. ,. What do you think of milk?
- It's really good for our



In [16]:
# Example 2: Factual question
response = generate_response(
    "What is the capital of France?",
    max_tokens=100,
    temperature=0.7
)

Prompt: What is the capital of France?
Response: <|user_end|><|assistant_start|>The capital of France is Paris.<|assistant_end|>



In [17]:
# Example 3: Math problem
response = generate_response(
    "If I have 5 apples and buy 3 more, how many apples do I have?",
    max_tokens=100,
    temperature=0.7
)

Prompt: If I have 5 apples and buy 3 more, how many apples do I have?
Response: <|user_end|><|assistant_start|>With 5 apples, I have 5 + 3 = <|python_start|>5+3<|python_end|><|output_start|>8<|output_end|>8 apples.
Therefore, the total number of apples is 8 + 5 = <|python_start|>8+5<|python_end|><|output_start|>13<|output_end|>13.
#### 13<|assistant_end|>



In [18]:
# Example 4: Creative writing
response = generate_response(
    "Write a short poem about artificial intelligence.",
    max_tokens=150,
    temperature=0.9
)

Prompt: Write a short poem about artificial intelligence.
Response: <|user_end|><|assistant_start|>Deep within a fog-shrouded city, where machines whisper secrets, a spark ignited 
The first AI system, a creation of human hands divine 
A world where machines rule, and minds take their share<|assistant_end|>



In [19]:
# Example 5: Coding question
response = generate_response(
    "Explain what a for loop does in Python.",
    max_tokens=150,
    temperature=0.7
)

Prompt: Explain what a for loop does in Python.
Response: <|user_end|><|assistant_start|>In Python, a `for` loop is a control structure that allows you to execute a block of code repeatedly for a specified number of iterations. It consists of three main parts:

1. **Initialization**: The loop starts with an initial value (`i`) and is executed using the `for` keyword.
2. **Control flow**: The loop continues until a `break` statement is encountered, at which point the execution is stopped and the loop body is returned.

The general syntax for a `for` loop is:
```python
for variable in iterable:
    # code to be executed
```

Here's a breakdown of the components:

- `i`:



## 8. Interactive Chat Mode

Create an interactive chat interface where you can have ongoing conversations.

In [ ]:
def chat_interactive():
    """
    Simple interactive chat mode.
    Type 'quit' or 'exit' to end the conversation.
    """
    print("=" * 60)
    print("NanoChat Interactive Mode")
    print("Type 'quit' or 'exit' to end the conversation")
    print("=" * 60)
    print()

    while True:
        # Get user input
        user_input = input("You: ")

        # Check for exit commands
        if user_input.lower() in ['quit', 'exit']:
            print("Goodbye!")
            break

        # Generate response
        print("NanoChat: ", end="", flush=True)
        response = generate_response(
            user_input,
            max_tokens=200,
            temperature=0.8,
            verbose=False
        )
        print(response)
        print()

# Uncomment to start interactive mode
# chat_interactive()

## 9. Experiment with Generation Parameters

Try different parameters to see how they affect the output.

In [ ]:
prompt = "Tell me an interesting fact about space."

print("Testing different temperature settings:\n")

# Low temperature (more deterministic)
print("Temperature = 0.3 (Conservative):")
generate_response(prompt, max_tokens=80, temperature=0.3)

print("\n" + "="*60 + "\n")

# Medium temperature (balanced)
print("Temperature = 0.7 (Balanced):")
generate_response(prompt, max_tokens=80, temperature=0.7)

print("\n" + "="*60 + "\n")

# High temperature (more creative)
print("Temperature = 1.0 (Creative):")
generate_response(prompt, max_tokens=80, temperature=1.0)

## 10. Batch Inference

Process multiple prompts efficiently.

In [ ]:
# List of prompts to process
prompts = [
    "What is machine learning?",
    "Explain quantum computing in simple terms.",
    "What are the benefits of renewable energy?",
    "How do neural networks work?"
]

print("Processing batch of prompts...\n")
print("="*60)

for i, prompt in enumerate(prompts, 1):
    print(f"\nPrompt {i}/{len(prompts)}: {prompt}")
    print("-"*60)
    response = generate_response(
        prompt,
        max_tokens=100,
        temperature=0.7,
        verbose=False
    )
    print(f"Response: {response}")
    print("="*60)

## 11. Custom Prompt Testing

Use this cell to test your own prompts!

In [ ]:
# Customize these parameters
YOUR_PROMPT = "What is the meaning of life?"
MAX_TOKENS = 200
TEMPERATURE = 0.8
TOP_K = 50

# Generate response
response = generate_response(
    YOUR_PROMPT,
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_k=TOP_K
)

##11. Testing Python Tool Usage

The nanochat model has been trained with Python code execution capabilities using special tokens:

<|python_start|> ... <|python_end|> - Python code
<|output_start|> ... <|output_end|> - Execution output
Let's test different prompting strategies to encourage the model to use Python correctly.

In [20]:
# Test 1: Explicit Python instruction
print("\n🧪 Test 1: Explicit Python instruction")
generate_response(
    "If I have 5 apples and buy 3 more, how many apples do I have? Use Python to calculate.",
    max_tokens=150,
    temperature=0.7
);


🧪 Test 1: Explicit Python instruction
Prompt: If I have 5 apples and buy 3 more, how many apples do I have? Use Python to calculate.
Response: <|user_end|><|assistant_start|>5 apples is equal to 5 x 3 = <|python_start|>5*3<|python_end|><|output_start|>15<|output_end|>15.
Therefore, you have 15 - 7 = <|python_start|>15-7<|python_end|><|output_start|>8<|output_end|>8 apples left.
#### 8<|assistant_end|>



In [21]:
# Test 2: "Calculate" keyword
print("\n🧪 Test 2: Using 'Calculate' keyword")
generate_response(
    "Calculate: 5 + 3",
    max_tokens=100,
    temperature=0.7
);


🧪 Test 2: Using 'Calculate' keyword
Prompt: Calculate: 5 + 3
Response:  * 2 = <|python_start|>5+3*2<|python_end|><|output_start|>11<|output_end|>11

#### 11<|assistant_end|>



In [22]:
# Test 3: "Compute" keyword
print("\n🧪 Test 3: Using 'Compute' keyword")
generate_response(
    "Compute the result of 5 apples plus 3 apples.",
    max_tokens=100,
    temperature=0.7
);


🧪 Test 3: Using 'Compute' keyword
Prompt: Compute the result of 5 apples plus 3 apples.
Response: <|user_end|><|assistant_start|>5 apples plus 3 apples = <|python_start|>5+3<|python_end|><|output_start|>8<|output_end|>8 apples.
Therefore, the total number of apples is 8 * 2 = <|python_start|>8*2<|python_end|><|output_start|>16<|output_end|>16.
#### 16<|assistant_end|>



In [23]:
# Test 3: "Compute" keyword
print("\n🧪 Test 3: Using 'Compute' keyword")
generate_response(
    "Compute the result of 5 apples plus 3 apples.",
    max_tokens=100,
    temperature=0.7
);


🧪 Test 3: Using 'Compute' keyword
Prompt: Compute the result of 5 apples plus 3 apples.
Response: <|user_end|><|assistant_start|>5 apples plus 3 apples = <|python_start|>5+3<|python_end|><|output_start|>8<|output_end|>8 apples.
Therefore, the total number of apples is 8 * 2 = <|python_start|>8*2<|python_end|><|output_start|>16<|output_end|>16.
#### 16<|assistant_end|>



In [24]:
# Test 6: Lower temperature for more deterministic behavior
print("\n🧪 Test 6: Lower temperature (0.3) with explicit instruction")
generate_response(
    "Use Python to calculate: 5 + 3",
    max_tokens=100,
    temperature=0.3  # Lower temperature for more consistent tool use
);


🧪 Test 6: Lower temperature (0.3) with explicit instruction
Prompt: Use Python to calculate: 5 + 3
Response:  + 2 + 1 = <|python_start|>5+3+2+1<|python_end|><|output_start|>11<|output_end|>11.
#### 11<|assistant_end|>



In [27]:
# Test 7: GSM8K-style problem (the model was trained on these)
print("\n🧪 Test 7: GSM8K-style word problem")
generate_response(
    """Q: Janet has 5 apples. She buys 3 more apples at the store.
How many apples does Janet have now?
A: Let's solve this step by step.""",
    max_tokens=150,
    temperature=0.1
);


🧪 Test 7: GSM8K-style word problem
Prompt: Q: Janet has 5 apples. She buys 3 more apples at the store.
How many apples does Janet have now?
A: Let's solve this step by step.
Response:  First find the total number of apples: 5 apples + 3 apples = <|python_start|>5+3<|python_end|><|output_start|>8<|output_end|>8 apples.
Then find the difference: 8 - 3 = <|python_start|>8-3<|python_end|><|output_start|>5<|output_end|>5 apples.
#### 5<|assistant_end|>



In [29]:
# Test 7: GSM8K-style problem (the model was trained on these)
print("\n🧪 Test 7: GSM8K-style word problem")
generate_response(
  """Q: Janet has 5 apples. She buys 3 more apples at the store.
How many apples does Janet have now?
A: Janet starts with 5 apples and buys 3 more.
Total = 5 + 3 = """,
    max_tokens=150,
    temperature=0.1
);


🧪 Test 7: GSM8K-style word problem
Prompt: Q: Janet has 5 apples. She buys 3 more apples at the store.
How many apples does Janet have now?
A: Janet starts with 5 apples and buys 3 more.
Total = 5 + 3 = 
Response: <|python_start|>5+3<|python_end|><|output_start|>8<|output_end|>8 apples.
Therefore, Janet has 8 - 5 = <|python_start|>8-5<|python_end|><|output_start|>3<|output_end|> Scotts.
#### 3<|assistant_end|>



## 📝 Notes and Tips

### Model Characteristics:
- **Size**: ~1.9B parameters (relatively small by modern standards)
- **Strengths**: Conversational tasks, general knowledge, educational purposes
- **Limitations**: May hallucinate, limited knowledge cutoff, not suitable for production critical applications

### Generation Parameters:
- **temperature**: Controls randomness (0.0 = deterministic, 1.0+ = creative)
  - Low (0.3-0.5): More focused and consistent responses
  - Medium (0.7-0.8): Balanced creativity and coherence
  - High (0.9-1.2): More creative but potentially less coherent

- **top_k**: Limits sampling to top K probable tokens
  - Lower values (20-30): More conservative
  - Higher values (50-100): More diverse

- **max_tokens**: Maximum number of tokens to generate
  - Adjust based on desired response length

### Performance Tips:
1. Use GPU runtime for faster inference (Runtime > Change runtime type > T4 GPU)
2. Adjust batch size if processing multiple prompts
3. Lower temperature for factual questions
4. Higher temperature for creative tasks

### Resources:
- [NanoChat GitHub Repository](https://github.com/karpathy/nanochat)
- [Model on Hugging Face](https://huggingface.co/jasonacox/nanochat-1.8B-rl)
- [Andrej Karpathy's Discussion](https://github.com/karpathy/nanochat/discussions)

---

## 🎉 You're all set!

This notebook demonstrates the basic usage of the nanochat-1.8B-rl model. Feel free to experiment with different prompts and parameters to explore the model's capabilities.

**Remember**: This is a micro-model designed for educational purposes. While it can generate interesting responses, it's not as capable as larger commercial models like GPT-4 or Claude.

Happy experimenting! 🚀